In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2
import math

In [2]:
# PrivateTest 3589, PublicTest 3589, Training 28709, 35887 total
def csv_reading(f_name):
    with open(f_name, 'r') as data:
        temp = pd.read_csv(data)
    temp = temp.values.tolist()
    for i in range(len(temp)):
        temp[i][1] = temp[i][1].split(' ')
    images = [item[1] for item in temp]
    for i in range(len(images)):
        images[i] = np.asarray([int(x) for x in images[i]])
        images[i] = images[i].reshape([48, 48, 1])
    labels = np.asarray([item[0] for item in temp])   
    return np.asarray(images), labels

In [3]:
images, labels = csv_reading('fer2013.csv')

In [4]:
# cv2.imwrite('test1.jpg', images[2])

In [5]:
train_img = images[:32298]
test_img = images[32298:]
train_labels = labels[:32298]
test_labels = labels[32298:]


In [6]:
train_tensor = tf.convert_to_tensor(train_img, dtype=tf.float32)
labels_tensor = tf.convert_to_tensor(train_labels, dtype=tf.int64)

train_slices = tf.data.Dataset.from_tensor_slices(train_tensor)
labels_slices = tf.data.Dataset.from_tensor_slices(labels_tensor)

batch_size = 128
train_batch = train_slices.batch(batch_size).repeat()
labels_batch = labels_slices.batch(batch_size).repeat()

train_iter = train_batch.make_initializable_iterator()
labels_iter = labels_batch.make_initializable_iterator()

next_train = train_iter.get_next()
next_label = labels_iter.get_next()

In [7]:
input_img = tf.placeholder(tf.float32, shape=(None, 48, 48, 1))
input_labels = tf.placeholder(tf.int64, shape=None)

In [8]:
regularizer = tf.contrib.layers.l2_regularizer(scale=1e-7)

In [9]:
def cnn_fc_model(input_img):
    with tf.variable_scope('conv1'):
        conv1 = tf.layers.conv2d(input_img, 64, (3, 3), 
                                 strides=(1, 1), 
                                 padding='same', 
                                 activation=None, 
                                 name='conv1', 
                                 reuse=False,
                                 kernel_regularizer=regularizer)
        
        conv1 = tf.layers.batch_normalization(conv1, training=True, name='BN1', reuse=False)
        conv1 = tf.nn.relu(conv1, name='relu1')      
        conv1 = tf.layers.max_pooling2d(conv1, (2, 2), (1, 1), padding='same', name='max_p1')
        conv1 = tf.nn.dropout(conv1, 0.5, name='dropout1') 
    with tf.variable_scope('conv2'):
        conv2 = tf.layers.conv2d(conv1, 128, (5, 5), 
                                 strides=(1, 1), 
                                 padding='same', 
                                 activation=None, 
                                 name='conv2', 
                                 reuse=False,
                                 kernel_regularizer=regularizer)
        
        conv2 = tf.layers.batch_normalization(conv2, training=True, name='BN2', reuse=False)
        conv2 = tf.nn.relu(conv2, name='relu2')      
        conv2 = tf.layers.max_pooling2d(conv2, (2, 2), (2, 2), padding='same', name='max_p2')
        conv2 = tf.nn.dropout(conv2, 0.5, name='dropout2') 
    with tf.variable_scope('conv3'):
        conv3 = tf.layers.conv2d(conv2, 512, (3, 3), 
                                 strides=(1, 1), 
                                 padding='same', 
                                 activation=None, 
                                 name='conv3', 
                                 reuse=False,
                                 kernel_regularizer=regularizer)
        
        conv3 = tf.layers.batch_normalization(conv3, training=True, name='BN3', reuse=False)
        conv3 = tf.nn.relu(conv3, name='relu3')       
        conv3 = tf.layers.max_pooling2d(conv3, (2, 2), (2, 2), padding='same', name='max_p3')
        conv3 = tf.nn.dropout(conv3, 0.5, name='dropout3')
    with tf.variable_scope('conv4'):
        conv4 = tf.layers.conv2d(conv3, 512, (3, 3), 
                                 strides=(1, 1), 
                                 padding='same', 
                                 activation=None, 
                                 name='conv4', 
                                 reuse=False,
                                 kernel_regularizer=regularizer)
        
        conv4 = tf.layers.batch_normalization(conv4, training=True, name='BN4', reuse=False)
        conv4 = tf.nn.relu(conv4, name='relu4')       
        conv4 = tf.layers.max_pooling2d(conv4, (2, 2), (2, 2), padding='same', name='max_p4') 
        conv4 = tf.nn.dropout(conv4, 0.5, name='dropout4')
        conv4 = tf.layers.Flatten()(conv4)
    with tf.variable_scope('fc1'):
        fc1 = tf.contrib.layers.fully_connected(conv4, 256, activation_fn=None, weights_regularizer=regularizer)
        fc1 = tf.layers.batch_normalization(fc1, training=True, name='BN5', reuse=False)
        fc1 = tf.nn.relu(fc1, name='relu5')
        fc1 = tf.nn.dropout(fc1, 0.5, name='dropout5')
    with tf.variable_scope('fc2'):
        fc2 = tf.contrib.layers.fully_connected(fc1, 512, activation_fn=None, weights_regularizer=regularizer)
        fc2 = tf.layers.batch_normalization(fc2, training=True, name='BN6', reuse=False)
        fc2 = tf.nn.relu(fc2, name='relu6')
        fc2 = tf.nn.dropout(fc2, 0.5, name='dropout6')
    result = tf.contrib.layers.fully_connected(fc2, 7, activation_fn=None)
    return result

In [9]:
def shallow_cnn_fc_model(input_img):
    with tf.variable_scope('shallow_conv1'):
        conv1 = tf.layers.conv2d(input_img, 32, (3, 3), 
                                 strides=(1, 1), 
                                 padding='same', 
                                 activation=None, 
                                 name='conv1', 
                                 reuse=False,
                                 kernel_regularizer=regularizer)
        
        conv1 = tf.layers.batch_normalization(conv1, training=True, name='BN1', reuse=False)
        conv1 = tf.nn.relu(conv1, name='relu1')
        conv1 = tf.nn.dropout(conv1, 0.5, name='dropout1')       
    with tf.variable_scope('shallow_conv2'):
        conv2 = tf.layers.conv2d(conv1, 64, (3, 3), 
                                 strides=(1, 1), 
                                 padding='same', 
                                 activation=None, 
                                 name='conv2', 
                                 reuse=False,
                                 kernel_regularizer=regularizer)
        
        conv2 = tf.layers.batch_normalization(conv2, training=True, name='BN2', reuse=False)
        conv2 = tf.nn.relu(conv2, name='relu2')      
        conv2 = tf.layers.max_pooling2d(conv2, (2, 2), (2, 2), padding='same', name='max_p2')
        conv2 = tf.nn.dropout(conv2, 0.5, name='dropout2') 
        conv2 = tf.layers.Flatten()(conv2)
    with tf.variable_scope('shallow_fc1'):
        fc1 = tf.contrib.layers.fully_connected(conv2, 512, activation_fn=None, weights_regularizer=regularizer)
        fc1 = tf.layers.batch_normalization(fc1, training=True, name='BN3', reuse=False)
        fc1 = tf.nn.relu(fc1, name='relu3')
        fc1 = tf.nn.dropout(fc1, 0.5, name='dropout3')
    result = tf.contrib.layers.fully_connected(fc1, 7, activation_fn=None)
    return result

In [10]:
# tf.reset_default_graph()
# result_logits = shallow_cnn_fc_model(input_img)
result_logits = cnn_fc_model(input_img)

In [11]:
softmax_losses = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=input_labels, logits=result_logits))
softmax = tf.nn.softmax(result_logits)

adamoptimizer = tf.train.AdamOptimizer(learning_rate=0.01, beta1=0.9, beta2=0.999, epsilon=1e-08)
train_op = adamoptimizer.minimize(softmax_losses)

In [12]:
def accuracy_compute(softmax_result, labels):
    prediction = tf.argmax(softmax_result, axis=1)
    correct = tf.equal(prediction, labels)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    return accuracy  

In [13]:
accuracy = accuracy_compute(softmax, input_labels)

In [14]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
sess.run([train_iter.initializer, labels_iter.initializer])

[None, None]

In [15]:
accuracy_of_train_plot = np.array([])
loss_of_train_plot = np.array([])
accuracy_of_test_plot =np.array([])
loss_of_test_plot = np.array([])

In [21]:
# 1 2 5 5 
for epoch in range(17):      
# for step in range(1000):
    for step in range(math.ceil(len(train_img)/batch_size)):
        temp_input = sess.run(next_train)
        temp_label = sess.run(next_label)
        _, temp_loss, temp_accuracy = sess.run([train_op, softmax_losses, accuracy], 
                                               feed_dict={input_img: temp_input, input_labels: temp_label})
        # sess.run(train_op, feed_dict={input_img: temp_input, input_labels: temp_label})
        accuracy_of_train_plot = np.append(accuracy_of_train_plot, temp_accuracy)
        loss_of_train_plot = np.append(loss_of_train_plot, temp_loss)
        print('epoch ', epoch+1 ,' step ', step+1, ', loss is  ', temp_loss, '  accuracy is  ', temp_accuracy)
    
    current_loss, current_accuracy = sess.run([softmax_losses, accuracy], 
                                          feed_dict={input_img: test_img, input_labels: test_labels})
    print( 'epoch ', epoch+1 ,', the current loss on test set is ', current_loss, '  , accuracy is  ', current_accuracy)
    accuracy_of_test_plot = np.append(accuracy_of_test_plot, current_accuracy)
    loss_of_test_plot = np.append(loss_of_test_plot, current_loss)

epoch  1  step  1 , loss is   1.09439   accuracy is   0.546875
epoch  1  step  2 , loss is   1.10025   accuracy is   0.570312
epoch  1  step  3 , loss is   1.07452   accuracy is   0.609375
epoch  1  step  4 , loss is   1.05415   accuracy is   0.601562
epoch  1  step  5 , loss is   1.10411   accuracy is   0.554688
epoch  1  step  6 , loss is   1.14217   accuracy is   0.578125
epoch  1  step  7 , loss is   1.22073   accuracy is   0.539062
epoch  1  step  8 , loss is   1.07983   accuracy is   0.554688
epoch  1  step  9 , loss is   1.00349   accuracy is   0.671875
epoch  1  step  10 , loss is   1.0003   accuracy is   0.609375
epoch  1  step  11 , loss is   0.985046   accuracy is   0.625
epoch  1  step  12 , loss is   1.0751   accuracy is   0.554688
epoch  1  step  13 , loss is   1.16635   accuracy is   0.507812
epoch  1  step  14 , loss is   1.12165   accuracy is   0.570312
epoch  1  step  15 , loss is   1.12029   accuracy is   0.632812
epoch  1  step  16 , loss is   1.06185   accuracy is 

epoch  1  step  129 , loss is   1.12774   accuracy is   0.585938
epoch  1  step  130 , loss is   1.01   accuracy is   0.617188
epoch  1  step  131 , loss is   0.988881   accuracy is   0.609375
epoch  1  step  132 , loss is   0.891   accuracy is   0.640625
epoch  1  step  133 , loss is   0.967102   accuracy is   0.601562
epoch  1  step  134 , loss is   0.992815   accuracy is   0.570312
epoch  1  step  135 , loss is   1.08881   accuracy is   0.570312
epoch  1  step  136 , loss is   0.914046   accuracy is   0.679688
epoch  1  step  137 , loss is   1.0852   accuracy is   0.578125
epoch  1  step  138 , loss is   0.977106   accuracy is   0.625
epoch  1  step  139 , loss is   0.914456   accuracy is   0.65625
epoch  1  step  140 , loss is   1.01448   accuracy is   0.617188
epoch  1  step  141 , loss is   0.921913   accuracy is   0.625
epoch  1  step  142 , loss is   1.17751   accuracy is   0.53125
epoch  1  step  143 , loss is   1.02693   accuracy is   0.625
epoch  1  step  144 , loss is   0.8

epoch  2  step  1 , loss is   1.02567   accuracy is   0.585938
epoch  2  step  2 , loss is   1.05848   accuracy is   0.570312
epoch  2  step  3 , loss is   0.971506   accuracy is   0.640625
epoch  2  step  4 , loss is   1.05134   accuracy is   0.617188
epoch  2  step  5 , loss is   1.03851   accuracy is   0.5625
epoch  2  step  6 , loss is   0.900407   accuracy is   0.59375
epoch  2  step  7 , loss is   1.08718   accuracy is   0.578125
epoch  2  step  8 , loss is   1.02686   accuracy is   0.664062
epoch  2  step  9 , loss is   1.11023   accuracy is   0.632812
epoch  2  step  10 , loss is   1.07118   accuracy is   0.625
epoch  2  step  11 , loss is   1.10631   accuracy is   0.53125
epoch  2  step  12 , loss is   0.978281   accuracy is   0.65625
epoch  2  step  13 , loss is   1.04709   accuracy is   0.609375
epoch  2  step  14 , loss is   0.936963   accuracy is   0.632812
epoch  2  step  15 , loss is   0.926203   accuracy is   0.65625
epoch  2  step  16 , loss is   1.0181   accuracy is  

epoch  2  step  129 , loss is   1.02566   accuracy is   0.609375
epoch  2  step  130 , loss is   1.05049   accuracy is   0.570312
epoch  2  step  131 , loss is   1.02391   accuracy is   0.601562
epoch  2  step  132 , loss is   0.911763   accuracy is   0.664062
epoch  2  step  133 , loss is   0.935366   accuracy is   0.65625
epoch  2  step  134 , loss is   1.00992   accuracy is   0.570312
epoch  2  step  135 , loss is   1.12095   accuracy is   0.578125
epoch  2  step  136 , loss is   0.932554   accuracy is   0.648438
epoch  2  step  137 , loss is   1.11945   accuracy is   0.554688
epoch  2  step  138 , loss is   1.07338   accuracy is   0.570312
epoch  2  step  139 , loss is   0.818061   accuracy is   0.710938
epoch  2  step  140 , loss is   0.933214   accuracy is   0.640625
epoch  2  step  141 , loss is   0.96611   accuracy is   0.648438
epoch  2  step  142 , loss is   1.11038   accuracy is   0.585938
epoch  2  step  143 , loss is   1.00055   accuracy is   0.601562
epoch  2  step  144 ,

epoch  3  step  1 , loss is   1.02563   accuracy is   0.578125
epoch  3  step  2 , loss is   1.0541   accuracy is   0.523438
epoch  3  step  3 , loss is   0.931102   accuracy is   0.632812
epoch  3  step  4 , loss is   1.12142   accuracy is   0.601562
epoch  3  step  5 , loss is   0.948509   accuracy is   0.664062
epoch  3  step  6 , loss is   0.935781   accuracy is   0.671875
epoch  3  step  7 , loss is   0.973648   accuracy is   0.625
epoch  3  step  8 , loss is   0.998036   accuracy is   0.65625
epoch  3  step  9 , loss is   0.978636   accuracy is   0.6875
epoch  3  step  10 , loss is   1.02597   accuracy is   0.625
epoch  3  step  11 , loss is   1.04919   accuracy is   0.585938
epoch  3  step  12 , loss is   1.01582   accuracy is   0.617188
epoch  3  step  13 , loss is   1.15045   accuracy is   0.53125
epoch  3  step  14 , loss is   1.03476   accuracy is   0.625
epoch  3  step  15 , loss is   0.999026   accuracy is   0.640625
epoch  3  step  16 , loss is   1.12212   accuracy is   0

...

epoch  15  step  225 , loss is   0.679909   accuracy is   0.734375
epoch  15  step  226 , loss is   0.767619   accuracy is   0.765625
epoch  15  step  227 , loss is   0.66759   accuracy is   0.765625
epoch  15  step  228 , loss is   0.72174   accuracy is   0.765625
epoch  15  step  229 , loss is   0.826042   accuracy is   0.695312
epoch  15  step  230 , loss is   0.729651   accuracy is   0.703125
epoch  15  step  231 , loss is   0.954467   accuracy is   0.65625
epoch  15  step  232 , loss is   0.84344   accuracy is   0.679688
epoch  15  step  233 , loss is   0.732913   accuracy is   0.710938
epoch  15  step  234 , loss is   0.785866   accuracy is   0.710938
epoch  15  step  235 , loss is   0.698832   accuracy is   0.773438
epoch  15  step  236 , loss is   0.739997   accuracy is   0.757812
epoch  15  step  237 , loss is   0.727676   accuracy is   0.742188
epoch  15  step  238 , loss is   0.784733   accuracy is   0.71875
epoch  15  step  239 , loss is   0.751926   accuracy is   0.679688


epoch  16  step  97 , loss is   0.68602   accuracy is   0.734375
epoch  16  step  98 , loss is   0.890276   accuracy is   0.65625
epoch  16  step  99 , loss is   0.693191   accuracy is   0.78125
epoch  16  step  100 , loss is   0.935909   accuracy is   0.6875
epoch  16  step  101 , loss is   0.640064   accuracy is   0.757812
epoch  16  step  102 , loss is   0.598186   accuracy is   0.78125
epoch  16  step  103 , loss is   0.692428   accuracy is   0.78125
epoch  16  step  104 , loss is   0.597931   accuracy is   0.757812
epoch  16  step  105 , loss is   0.696601   accuracy is   0.734375
epoch  16  step  106 , loss is   0.668338   accuracy is   0.757812
epoch  16  step  107 , loss is   0.854123   accuracy is   0.695312
epoch  16  step  108 , loss is   0.702105   accuracy is   0.710938
epoch  16  step  109 , loss is   0.738107   accuracy is   0.75
epoch  16  step  110 , loss is   0.72396   accuracy is   0.734375
epoch  16  step  111 , loss is   0.820701   accuracy is   0.734375
epoch  16 

epoch  16  step  221 , loss is   0.707104   accuracy is   0.726562
epoch  16  step  222 , loss is   0.720497   accuracy is   0.734375
epoch  16  step  223 , loss is   0.801628   accuracy is   0.695312
epoch  16  step  224 , loss is   0.687533   accuracy is   0.71875
epoch  16  step  225 , loss is   0.589917   accuracy is   0.765625
epoch  16  step  226 , loss is   0.743961   accuracy is   0.765625
epoch  16  step  227 , loss is   0.661399   accuracy is   0.765625
epoch  16  step  228 , loss is   0.628726   accuracy is   0.773438
epoch  16  step  229 , loss is   0.801604   accuracy is   0.671875
epoch  16  step  230 , loss is   0.670988   accuracy is   0.726562
epoch  16  step  231 , loss is   0.754264   accuracy is   0.804688
epoch  16  step  232 , loss is   0.914106   accuracy is   0.648438
epoch  16  step  233 , loss is   0.762984   accuracy is   0.742188
epoch  16  step  234 , loss is   0.779135   accuracy is   0.75
epoch  16  step  235 , loss is   0.645107   accuracy is   0.773438


epoch  17  step  92 , loss is   0.785582   accuracy is   0.757812
epoch  17  step  93 , loss is   0.614813   accuracy is   0.757812
epoch  17  step  94 , loss is   0.656864   accuracy is   0.765625
epoch  17  step  95 , loss is   0.739892   accuracy is   0.71875
epoch  17  step  96 , loss is   0.830666   accuracy is   0.695312
epoch  17  step  97 , loss is   0.624577   accuracy is   0.789062
epoch  17  step  98 , loss is   0.757222   accuracy is   0.742188
epoch  17  step  99 , loss is   0.678513   accuracy is   0.703125
epoch  17  step  100 , loss is   0.779956   accuracy is   0.726562
epoch  17  step  101 , loss is   0.679923   accuracy is   0.734375
epoch  17  step  102 , loss is   0.554358   accuracy is   0.804688
epoch  17  step  103 , loss is   0.606714   accuracy is   0.78125
epoch  17  step  104 , loss is   0.6505   accuracy is   0.757812
epoch  17  step  105 , loss is   0.79066   accuracy is   0.71875
epoch  17  step  106 , loss is   0.713658   accuracy is   0.75
epoch  17  st

epoch  17  step  216 , loss is   0.774086   accuracy is   0.710938
epoch  17  step  217 , loss is   0.736359   accuracy is   0.703125
epoch  17  step  218 , loss is   0.66841   accuracy is   0.75
epoch  17  step  219 , loss is   0.64852   accuracy is   0.703125
epoch  17  step  220 , loss is   0.795831   accuracy is   0.703125
epoch  17  step  221 , loss is   0.688489   accuracy is   0.726562
epoch  17  step  222 , loss is   0.719553   accuracy is   0.75
epoch  17  step  223 , loss is   0.689765   accuracy is   0.75
epoch  17  step  224 , loss is   0.691812   accuracy is   0.734375
epoch  17  step  225 , loss is   0.686551   accuracy is   0.757812
epoch  17  step  226 , loss is   0.71589   accuracy is   0.742188
epoch  17  step  227 , loss is   0.753572   accuracy is   0.742188
epoch  17  step  228 , loss is   0.807726   accuracy is   0.71875
epoch  17  step  229 , loss is   0.780694   accuracy is   0.734375
epoch  17  step  230 , loss is   0.744914   accuracy is   0.710938
epoch  17  

In [20]:
current_loss, current_accuracy = sess.run([softmax_losses, accuracy], 
                                          feed_dict={input_img: test_img, input_labels: test_labels})
print('the current loss on test set is ', current_loss, '  accuracy is  ', current_accuracy)

the current loss on test set is  1.39649   accuracy is   0.489551


In [15]:
# writer = tf.summary.FileWriter('.')
# writer.add_graph(tf.get_default_graph())

In [22]:
np.savez('deep.npz', accuracy_of_train_plot = accuracy_of_train_plot, loss_of_train_plot = loss_of_train_plot, 
         accuracy_of_test_plot = accuracy_of_test_plot, loss_of_test_plot = loss_of_test_plot)

In [36]:
# Save the model
inputs = {'Input': input_img, 'Label': input_labels}
outputs = {'output': softmax_losses, 'accuracy': accuracy}
tf.saved_model.simple_save(sess, 'DeepCNN_Loss_Accuracy', inputs, outputs)

INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'DeepCNN_Loss_Accuracy/saved_model.pb'
